# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,282.11,86,0,1.05,ZA,1726517199
1,1,blackmans bay,-43.0167,147.3167,281.22,71,100,0.45,AU,1726517104
2,2,tobruk,32.0836,23.9764,298.39,56,0,2.37,LY,1726517201
3,3,albany,42.6001,-73.9662,301.99,42,36,0.45,US,1726517159
4,4,currumbin waters,-28.1553,153.4613,285.38,92,1,2.06,AU,1726517203


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
%%capture --no-display

# Configure the map plot
city_map_humidity = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles= "OSM", size="Humidity", frame_width=800, frame_height=600, title ="Cities and Humidity Map", color="City")

# Display the map
city_map_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
colder_cities_df = city_data_df[city_data_df["Max Temp"] <= 300]

# Drop any rows with null values
cleaned_cold_cities_df = colder_cities_df.dropna(how ="any")

# Display sample data
cleaned_cold_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,282.11,86,0,1.05,ZA,1726517199
1,1,blackmans bay,-43.0167,147.3167,281.22,71,100,0.45,AU,1726517104
2,2,tobruk,32.0836,23.9764,298.39,56,0,2.37,LY,1726517201
4,4,currumbin waters,-28.1553,153.4613,285.38,92,1,2.06,AU,1726517203
5,5,pisco,-13.7000,-76.2167,291.49,77,2,4.15,PE,1726517204


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_cold_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df.head()

C:\Users\Rowan\AppData\Local\Temp\ipykernel_1356\3871353731.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = " "


,City,Country,Lat,Lng,Humidity,Hotel Name
0,bredasdorp,ZA,-34.5322,20.0403,86,
1,blackmans bay,AU,-43.0167,147.3167,71,
2,tobruk,LY,32.0836,23.9764,56,
4,currumbin waters,AU,-28.1553,153.4613,92,
5,pisco,PE,-13.7000,-76.2167,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    #Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bredasdorp - nearest hotel: Victoria Hotel
blackmans bay - nearest hotel: Villa Howden
tobruk - nearest hotel: فندق زهرة البطنان -طبرق
currumbin waters - nearest hotel: Currumbin Sands Holiday Apartments
pisco - nearest hotel: La Portada
waitangi - nearest hotel: Hotel Chathams
carnarvon - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
grytviken - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
puerto natales - nearest hotel: Dorotea Patagonia Hostel
kodiak - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
wer - nearest hotel: No hotel found
tazovsky - nearest hotel: Тазовчанка
inongo - nearest hotel: Chez LELE IYOLO
la passe - nearest hotel: L'Estuaire
yellowknife - nearest hotel: No hotel found
dryden - nearest hotel: Holiday Inn Express
morogoro - nearest hotel: B One Lodge
badger - nearest hotel: Clarion Hotel & Suites
chui - nearest hotel: Alerce

coquimbo - nearest hotel: Hotel Iberia
ambodifotatra - nearest hotel: Les Palmiers
longjing - nearest hotel: 海兰江酒店
zonguldak - nearest hotel: No hotel found
jijiga - nearest hotel: Africa Hotel
villafranca tirrena - nearest hotel: Viola Palace Hotel
ulaangom - nearest hotel: АНДУУД
margate - nearest hotel: The Reading Rooms
maceio - nearest hotel: Maceió Hostel e Pousada
asubulaq - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
boryeong - nearest hotel: 리버사이드호텔
peace river - nearest hotel: New Ridge Inn
quellon - nearest hotel: Aliwen
monduli - nearest hotel: No hotel found
pipar - nearest hotel: Ganesh Hotel
woolgoolga - nearest hotel: Darlington Beach Resort
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
ballina - nearest hotel: Ballina Manor Boutique Hotel
port augusta - nearest hotel: Hotel Commonwealth
wynyard - nearest hotel: Wynyard Hotel
labasa - nearest hotel: Grand Eastern Hotel
fort bragg - nearest h

karacabey - nearest hotel: Ün Otel
rudnichnyy - nearest hotel: Общежитие
zambezi - nearest hotel: No hotel found
ginir - nearest hotel: No hotel found
kerikeri - nearest hotel: Avalon Resort
tamanrasset - nearest hotel: فندق الأمان
alesund - nearest hotel: Brosundet
plesetsk - nearest hotel: Плесецкая
pohrebyshche - nearest hotel: No hotel found
tchibanga - nearest hotel: Auberge Nyanga
narathiwat - nearest hotel: โรงแรมล่วนเฮง
zanjan - nearest hotel: هتل قصر
ash shafa - nearest hotel: No hotel found
dindima - nearest hotel: No hotel found
warrnambool - nearest hotel: Central Court
viedma - nearest hotel: Níjar
springs - nearest hotel: Manhattan Hotel & Action Bar
tarko-sale - nearest hotel: Вынгапур
guerrero negro - nearest hotel: Plaza sal paraiso
hami - nearest hotel: Хами
piacabucu - nearest hotel: O Leão
perbaungan - nearest hotel: No hotel found
phu ly - nearest hotel: Inco 515.9
tandil - nearest hotel: Viñas del Rosario Hotel Boutique


,City,Country,Lat,Lng,Humidity,Hotel Name
0,bredasdorp,ZA,-34.5322,20.0403,86,Victoria Hotel
1,blackmans bay,AU,-43.0167,147.3167,71,Villa Howden
2,tobruk,LY,32.0836,23.9764,56,فندق زهرة البطنان -طبرق
4,currumbin waters,AU,-28.1553,153.4613,92,Currumbin Sands Holiday Apartments
5,pisco,PE,-13.7000,-76.2167,77,La Portada
...,...,...,...,...,...,...
538,hami,CN,42.8000,93.4500,21,Хами
541,piacabucu,BR,-10.4056,-36.4344,74,O Leão
542,perbaungan,ID,3.5679,98.9560,93,No hotel found
543,phu ly,VN,20.5411,105.9139,91,Inco 515.9


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
%%capture --no-display

# Configure the map plot
city_hotels_map = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles= "OSM", size="Humidity", frame_width=800, frame_height=600, title ="Cities, Humidity and Hotels", color="City", hover_cols=["Country","Hotel Name"])


# Display the map
city_hotels_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)